In [ ]:
%matplotlib inline

## 2 - More on models and model description

This chapter will focus on the "mini-language" used to describe models in S-timator and also on the model objects created by function `read_model()`

In [ ]:
from stimator import read_model

Picking up the last example from the previous chapter: the _open two-enzyme system_:

![Example: a two-reaction open chemical system](images/2chem_open.png)

Recall that the minimum components of a model declaration are:

- title
- reactions
- parameters
- init

In [ ]:
model_description = """
title An open two-reaction chemical system

inflow: -> A, rate = kin
r1: A -> B, rate = k1 * A
r2: B -> C, rate = k2 * B - k3 * C
outflow: C ->, rate = kout * C

kin = 0.5
k1 = 0.1
k2 = 2
k3 = 1
kout = 0.2

init: (A = 0, B = 0, C = 0)
"""

m = read_model(model_description)

In [ ]:
print(m)

### 2.1 - Reactions

You can iterate over the reactions of a model:

In [ ]:
for v in m.reactions:
    print (v)

Or, just to get the names:

In [ ]:
for v in m.reactions:
    print (v.name)

A reaction has a lot of attributes:

In [ ]:
v = m.reactions.r1
print (v.name)
print (v.reagents)
print (v.products)
print (v.stoichiometry_string)
print (v.stoichiometry)
print (v())

`Model.varnames` is a list of variable names and `Model.parameters` can be used to iterate over the parameters of a model.

In [ ]:
print (m.varnames)
for p in m.parameters:
    print ('%6s = %g' % (p.name, p))

### 2.2 - Transformations

Transformations are declared starting a line with a `~`. These are quantities that vary over time but are not decribed by differential equations. In this example `total` is a transformation.

In [ ]:
model_description = """
title An open two-reaction chemical system

inflow: -> A, rate = kin
r1: A -> B, rate = k1 * A
r2: B -> C, rate = k2 * B - k3 * C
outflow: C ->, rate = kout * C

kin = 0.5
k1 = 0.1
k2 = 2
k3 = 1
kout = 0.2

init: (A = 0, B = 0, C = 0)

~ total = A + B + C
"""

m = read_model(model_description)
print(m)

In [ ]:
m.solve(tf=50.0, outputs=["total", 'A', 'B', 'C']).plot(show=True)

### 2.3 - Local parameters in processes

Parameters can also "belong", that is, being local, to processes. 

In this example, both `r1` and `r2` have local parameters

Notice how thes paraemters are listed and refered to in `print m`:

In [ ]:
model_description = """
title An open two-reaction chemical system

inflow: -> A, rate = kin
r1: A -> B, rate = k * A, k = 0.1
r2: B -> C, rate = kf * B - kr * C, kf = 2, kr = 1
outflow: C ->, rate = kout * C

kin = 0.5
kout = 0.2

init: (A = 0, B = 0, C = 0)

~ total = A + B + C
"""

m = read_model(model_description)
print(m)

But this model is exactly the same has the previous model. The parameters were just made local. (`plot()` produces the same results).

In [ ]:
m.solve(tf=50.0, outputs=["total", 'A', 'B', 'C']).plot()

The iteration of the parameters is now different:

In [ ]:
for p in m.parameters:
    print(p.name, p)

### 2.4 - External variables

An external variable is a parameter that appears in the stoichiometry of a reaction. It is treated as a constant.

In this example, `D` is an external variable:

In [ ]:
m = read_model("""
title An open two-reaction chemical system

inflow: D -> A, rate = kin * D
r1: A -> B, rate = k * A, k = 0.1
r2: B -> C, rate = kf * B - kr * C, kf = 2, kr = 1
outflow: C -> E, rate = kout * C

D = 1
kin = 0.5
kout = 0.2
E = 2

init: (A = 0, B = 0, C = 0)

~ total = A + B + C
""")
print(m)

In [ ]:
m.solve(tf=50.0, outputs=['A', 'B', 'C', 'D', 'E']).plot()

### 2.5 - Declaration of outputs

You can use `!!` to specify what should go into the solution of the model:

In [ ]:
m = read_model("""
title An open two-reaction chemical system

inflow: D -> A, rate = kin * D
r1: A -> B, rate = k * A, k = 0.1
r2: B -> C, rate = kf * B - kr * C, kf = 2, kr = 1
outflow: C -> E, rate = kout * C

D = 1
kin = 0.5
kout = 0.2
E = 2

init: (A = 0, B = 0, C = 0)

~ total = A + B + C

!! C D E
""")
m.solve(tf=50.0).plot()

Or use the `outputs` argument to the `solve()` function (in the form of a list of desired outputs):

In [ ]:
m.solve(tf=50.0, outputs=['total', 'A']).plot()

`->` can be used to specify the values of all the rates of all the processes.

In [ ]:
m.solve(tf=50.0, outputs=['->', 'D']).plot()

### 2.6 - Explicit differential equations

In [ ]:
m = read_model("""
title mass on a spring, frictionless

# F = m * a = m * v' = - k * x
# by Hooke's law and Newton's law of motion

v' = -(k * x) / m
x' = v

m = 0.5
k = 1

init: x = 1
""")
m.solve(tf=10.0).plot()

In [ ]:
m = read_model("""
title mass on a spring, with friction

# using Hooke's law and friction proportional to speed,
# F = m * a = m * v' = - k * x - b * v

v' = (-k*x - b*v) / m
x' = v

m = 0.5
k = 1
b = 0.5

init: x = 1
""")
m.solve(tf=10.0).plot()

### 2.7 - Forcing functions

In [ ]:
m = read_model("""
title An open two-reaction chemical system

inflow: D -> A, rate = kin * D * step(t, 10, 1)
r1: A -> B, rate = k * A, k = 0.1
r2: B -> C, rate = kf * B - kr * C, kf = 2, kr = 1
outflow: C -> E, rate = kout * C

D = 1
kin = 0.5
kout = 0.2
E = 2

init: (A = 0, B = 0, C = 0)

!! inflow A B C E
""")
m.solve(tf=50.0).plot()